In [9]:
import pandas as pd
from openai import OpenAI, Embedding
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import os


In [97]:
df = pd.read_csv("../20_11_26_11_articles_combined.csv") 
#openai_client = OpenAI(api_key = 'insert-api-key')

In [4]:
print(df.head())

           title                                                url  \
0  Yahoo Finance  https://finance.yahoo.com/news/veteran-fund-ma...   
1  Yahoo Finance  https://finance.yahoo.com/news/messenger-gets-...   
2  Yahoo Finance  https://finance.yahoo.com/news/social-media-ba...   
3  Yahoo Finance  https://finance.yahoo.com/news/snowflake-share...   
4  Yahoo Finance  https://finance.yahoo.com/news/baidu-bulls-ret...   

                          date  \
0   Wed, Nov 20, 2024, 3:33 PM   
1  Wed, Nov 20, 2024, 10:06 AM   
2   Wed, Nov 20, 2024, 5:00 PM   
3   Wed, Nov 20, 2024, 1:30 PM   
4   Wed, Nov 20, 2024, 4:28 PM   

                                             content  
0  We are experiencing some temporary issues. The...  
1  We are experiencing some temporary issues. The...  
2  We are experiencing some temporary issues. The...  
3  We are experiencing some temporary issues. The...  
4  We are experiencing some temporary issues. The...  


In [13]:
from openai import OpenAI, Embedding


def summarize_doc(openai_client, text):
    query = "Summarize this into less than 10 words"

    prompt = f"Context:\n{text}\n\nQuestion: {query}\nAnswer:"


    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
        {"role": "user", "content": prompt}
        ]
    )

    print(completion.choices[0].message)

In [68]:
def preprocess_text(text):
    """Preprocess a single text input (either document or query)."""
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')  # Add custom stop words if needed
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # Replace punctuation with spaces
    text_nopunct = text.translate(translator)  # Remove punctuation
    words_list = text_nopunct.strip().split()  # Tokenize into words
    filtered_words = [word.lower() for word in words_list if word.lower() not in stop_words and len(word) > 1]
    return filtered_words

def preprocess_df(df):
    """Preprocess a DataFrame column containing document content."""
    df["content"] = df["content"].apply(preprocess_text)
    return df

def preprocess_query(query):
    """Preprocess a query string."""
    return preprocess_text(query)

In [17]:
import bm25
import importlib

# Reload the module after making changes
importlib.reload(bm25)

# Initialize the BM25 instance

def rank_query_bm25(bm25, query, top_k=5, summarize_docs=True):
    # Display initial document details
    print("=" * 50)
    print("📄 Document Sample:")
    print(df.head())
    print("=" * 50)
    print(f"📏 Total Documents: {len(bm25_instance.doc_lengths)}")
    print(f"Preprocessed query: {query}")
    # Rank documents for a given query
    ranked_indices, scores = bm25_instance.rank(query)

    ranked_indices = ranked_indices[:top_k]
    scores = scores[:top_k]

    print("\n🏆 Ranking Results:")
    print("=" * 50)
    for i, (doc_index, score) in enumerate(zip(ranked_indices, scores), start=1):
        print(f"⭐ Rank {i}:")
        print(f"   - Document Index: {doc_index}")
        print(f"   - BM25 Score: {score:.4f}")
        doc_content = df.iloc[doc_index]["content"]
        if (summarize_docs):
            summmary = summarize_doc(openai_client, doc_content)
            print(f"   - Document Content:\n{summmary}")
        else:
            print(f"   - Document Content:\n{doc_content}")
        print("-" * 50)


In [94]:
query = "Why is Australia's monetary policy still seen as powerful even though a lot of households have high mortgage debt?" #408
#28 15
query2 = "What's Palo Alto Networks' revenue forecast for the second quarter, and how does it stack up against what analysts expected?" #34
#35 18
query3= "How could institutional investments affect Bitcoin's price through 2025?" #229
#18 13
query4 = "Hey, I've been hearing that some AI companies are getting into hardware. What challenges do these companies face when trying to bring AI devices to market?" #45
#21.3 20.7
query5 = "What impact could increased tariffs have on the cost of goods for companies, and how might that affect consumers?" #373
#17.4 15.6
query6 = "Why is the U.S. government reducing Intel's chips grant, and what factors are contributing to the change?" #191
#21.9 20.8
query7 = "What is Israel considering regarding a cease-fire with Hezbollah, and what would the terms be?" #12
#29 23
query8 = "How have Baidu's shares performed this year, and what’s the outlook for its upcoming earnings report?" #222
#13.9 13.1
query9 = "What challenges does the U.S. face in reaching an agreement over the future of Colorado River water use, and what are the possible solutions being discussed?" #411
#32 11
query10 = "Why are creditors frustrated with the stalled auction of Citgo Petroleum shares?" #91
#35 8

query = query6

In [78]:
df = preprocess_df(df)

In [95]:
query = preprocess_query(query)
bm25_instance = bm25.BM25(df["content"],k1=1.5, b=0.75)
rank_query_bm25(bm25_instance, query, summarize_docs=False)


📄 Document Sample:
           title                                                url  \
0  Yahoo Finance  https://finance.yahoo.com/news/canada-oil-patc...   
1  Yahoo Finance  https://finance.yahoo.com/news/thanksgiving-di...   
2  Yahoo Finance  https://finance.yahoo.com/news/amgen-falls-wei...   
3  Yahoo Finance  https://finance.yahoo.com/news/ken-leech-sued-...   
4  Yahoo Finance  https://finance.yahoo.com/news/chocolatiers-bu...   

                         date  \
0  Tue, Nov 26, 2024, 8:57 AM   
1  Tue, Nov 26, 2024, 9:32 AM   
2  Tue, Nov 26, 2024, 6:38 AM   
3  Tue, Nov 26, 2024, 7:57 AM   
4  Tue, Nov 26, 2024, 7:00 AM   

                                             content  
0  [bloomberg, canada’s, oil, industry, stressed,...  
1  [count, blessings, thanksgiving, dinner, cheap...  
2  [bloomberg, amgen, inc, shares, fell, 23, year...  
3  [bloomberg, first, blush, us, case, star, bond...  
4  [bloomberg, looked, like, cocoa, supplies, imp...  
📏 Total Documents: 437
Pr

In [5]:
count=0
for i in range(len(df)):
    article = df.iloc[i]["content"]
    if len(article) < 100:
        count+=1
        print(article)
print(count)

0


In [98]:
df.iloc[91]["content"]

'By Gary McWilliams and Marianna Parraga HOUSTON (Reuters) - Court advisers have billed nearly $30 million for a stalled auction of shares in a parent of Venezuela-owned oil refiner Citgo Petroleum, raising the ire of creditors that have waited years to get compensation. Citgo, the crown jewel of Venezuela\'s overseas assets, sits at the center of a Delaware court auction in which 18 companies seek to collect up to $21.3 billion for debt defaults and expropriations in the South American country. Bids in the auction\'s second round were submitted this year, and the advisers, including an officer appointed by the court to oversee the process, were to deliver a recommended winner in July. The advisers went on to negotiate exclusively with an affiliate of investor Elliott Investment Management, which has resulted in a bid opposed by many creditors as deficient. Four creditors in a court filing challenged the advisers\' $4.1 million bill for September, saying fees "have increased by a stagg